In [1]:
import numpy as np

In [59]:

class Quaternion(np.ndarray):
    """Only works with unit quaternions (as needed to describe rotations)"""
    
    def __new__(cls, input_array=[0.,1.,0.,0.]):
        """By default works with 0 rotation about x axis"""
        input_array = np.array(input_array)/np.linalg.norm(input_array)
        
        obj = np.asarray(input_array).view(cls)
        
        return obj
    
    def __array__finalize(self, obj):
        pass

    
    @classmethod
    def from_angle(cls, theta, axis):
        """A rotation of angle `theta` about the axis `ax, ay, az`. Allows the axis to be not normalized"""

        axis = np.array(axis)/np.linalg.norm(axis)

        s = np.cos(theta / 2)
        v = np.sin(theta / 2) * axis

        return cls([s, *v])

    def normalize(self):
        """Convert quaternion to unit vector"""
        
        self/np.linalg.norm(self)
        
    def rot_matrix(self):
        """Generate a rotation matrix"""

        R = np.array([[1 - 2 * self[2]**2 - 2 * self[3]**2,
                       2 * self[1] * self[2] - 2 * self[0] * self[3],
                       2 * self[1] * self[3] + 2 * self[0] * self[2]],
                [2 * self[1] * self[2] + 2 * self[0] * self[3],
                 1 - 2 * self[1]**2 - 2 * self[3]**2,
                 2 * self[2] * self[3] - 2 * self[0] * self[1]],
                [2 * self[1] * self[3] - 2 * self[0] * self[2],
                 2 * self[2] * self[3] + 2 * self[0] * self[1],
                 1 - 2 * self[1]**2 - 2 * self[2]**2]])

        return R

    def rate_of_change(self, omega):
        """Return the rate of change of the quaternion based on an angular velocity"""
        
        # follows the formulation in Box
        # note that there are other methods, which may be more accurate
        
        omega = np.array(omega)
        
        s = self[0]
        v = np.array(self[1:])

        sdot = - 0.5 * (omega @ v) # mistake in Box eqn 7 (based on https://www.sciencedirect.com/topics/computer-science/quaternion-multiplication and http://web.cs.iastate.edu/~cs577/handouts/quaternion.pdf)
        vdot = 0.5 * (s * omega + np.cross(omega, v))

        # returns a quaternion object as a rate of change is really just a quaternion
        return Quaternion([sdot, *vdot])
    
    def __mul__(self, other):
        raise NotImplementedError


In [51]:
q = Quaternion()
q

Quaternion([0., 1., 0., 0.])

In [52]:
q.rot_matrix()

array([[ 1.,  0.,  0.],
       [ 0., -1.,  0.],
       [ 0.,  0., -1.]])

In [57]:
q.rate_of_change(omega=[0.,1,1])

Quaternion([-0.        ,  0.        ,  0.70710678, -0.70710678])

In [54]:
[0,1,1] @ np.array(q[1:])

0.0

In [55]:
q[0]

0.0